This box will set up the name of the slice and set the expected resources<br>

In [23]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

cores_per_node = 16
ram_per_node = 32
disk_per_node = 20 # we will modify this later to give fileservers more
site_name = fablib.get_random_site(
    filter_function=lambda x: x["cores_available"] > cores_per_node * 10 and x["ram_available"] > ram_per_node * 10 and x["disk_available"] > disk_per_node * 10
)


import os
slice_name="IP_NDN_Test_2_" + os.getenv('NB_USER')

try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
except:
    print("You don't have a slice named %s yet." % slice_name)
    slice = fablib.new_slice(name=slice_name)

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Bastion Username,sjzhu_0000105651
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


You don't have a slice named IP_NDN_Test_2_sjzhu yet.


<br><br>Next we name and set the networks and interfaces<br>

In [24]:
# this cell sets up the hosts and router
node_names = ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]
for n in node_names:
    node = slice.add_node(name=n, site=site_name, cores=16, ram=32, disk=20, image='default_ubuntu_22')
    node.add_post_boot_upload_directory("scripts")
    #node.add_post_boot_execute("./scripts/install_dependencies.sh")
    node.add_post_boot_execute("./scripts/install-ndn-dpdk.sh")
    
# this cell sets up the network links
nets = [
    {"name": "net1",   "nodes": ["r1","c1","c2"]},
    {"name": "net2",   "nodes": ["r2","p1"]},
    {"name": "net3",   "nodes": ["r3","p2"]},
    {"name": "net4",   "nodes": ["r4","p3","c3"]},
    {"name": "net12",   "nodes": ["r1","r2"]},
    {"name": "net23",   "nodes": ["r2","r3"]},
    {"name": "net34",   "nodes": ["r3","r4"]},
    {"name": "net41",   "nodes": ["r4","r1"]},
]
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

<br><br>Request the slice and wait until it has been reserved<br>

In [25]:
slice.submit()


Retry: 10, Time: 4060 sec


ID,3e42a97d-add1-4ec5-8c9c-f615afe41302
Name,IP_NDN_Test_2_sjzhu
Lease Expiration (UTC),2023-11-21 01:19:27 +0000
Lease Start (UTC),2023-11-20 01:19:28 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
35883164-5a73-43a4-858a-5fc80d4efb2b,c1,16,32,100,default_ubuntu_22,qcow2,losa-w2.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fea9:d421,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fea9:d421,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7896a74d-7e6a-4409-b35b-f7e9c34b9722,c2,16,32,100,default_ubuntu_22,qcow2,losa-w2.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe08:8f64,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe08:8f64,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
bc3b50b9-3f0c-42fc-b3ee-6e6bb6be3438,c3,16,32,100,default_ubuntu_22,qcow2,losa-w2.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fef3:9dc1,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fef3:9dc1,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e2a51f16-77c9-4d61-b06b-d2bb6f46bd06,p1,16,32,100,default_ubuntu_22,qcow2,losa-w2.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe6d:de91,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe6d:de91,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d4faad99-a7ce-4353-a6d7-57e29089e769,p2,16,32,100,default_ubuntu_22,qcow2,losa-w2.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe19:16ac,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe19:16ac,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
657e9d83-91e2-4de6-93d0-d0a649541d46,p3,16,32,100,default_ubuntu_22,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe1a:7763,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe1a:7763,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1b185555-006c-40f3-bc34-d69f359eecc5,r1,16,32,100,default_ubuntu_22,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe5f:5494,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe5f:5494,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c6948c83-de9a-47ad-a5ad-dc159b0fdf42,r2,16,32,100,default_ubuntu_22,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fec4:54fa,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fec4:54fa,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ea52e516-4099-4a6f-a4f5-a063b86a4b6f,r3,16,32,100,default_ubuntu_22,qcow2,losa-w1.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fed9:1c1e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fed9:1c1e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
57dacc96-f4c8-452c-8435-c61190e7ebe3,r4,16,32,100,default_ubuntu_22,qcow2,losa-w1.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fefb:11ed,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fefb:11ed,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/w

ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
e3d8884f-fc82-4447-97cd-8aeff5c54ab8,net1,L2,L2Bridge,LOSA,None,None,Active,
f819a3d4-5970-4d06-9875-abf7df066499,net12,L2,L2Bridge,LOSA,None,None,Active,
29aa0fcd-d85e-41e5-bb76-f8b07f718d60,net2,L2,L2Bridge,LOSA,None,None,Active,
aa9a59e2-2bbf-4874-89ae-f9ab35993220,net23,L2,L2Bridge,LOSA,None,None,Active,
500a56de-7c08-4a8f-95e5-ab5bdf728ab7,net3,L2,L2Bridge,LOSA,None,None,Active,
764f9979-3f38-44b0-bed1-520cdbf0d7e2,net34,L2,L2Bridge,LOSA,None,None,Active,
dba67253-414f-4101-af0a-d160a79709a2,net4,L2,L2Bridge,LOSA,None,None,Active,
d620262e-5288-4234-a1d7-bdefa73b7fad,net41,L2,L2Bridge,LOSA,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
c1-net1-p1,p1,c1,net1,100,config,,0A:4C:01:D1:D8:41,enp6s0,enp6s0,None,4
c2-net1-p1,p1,c2,net1,100,config,,0E:47:AA:3F:E1:AA,enp6s0,enp6s0,None,4
c3-net4-p1,p1,c3,net4,100,config,,12:CC:D9:CC:0E:2F,enp6s0,enp6s0,None,4
p1-net2-p1,p1,p1,net2,100,config,,0E:A4:9F:77:13:42,enp6s0,enp6s0,None,4
p2-net3-p1,p1,p2,net3,100,config,,12:FC:DC:55:68:10,enp6s0,enp6s0,None,4
p3-net4-p1,p1,p3,net4,100,config,,2E:4A:50:82:4A:96,None,None,None,4
r1-net1-p1,p1,r1,net1,100,config,,32:10:BC:55:EC:24,enp6s0,enp6s0,None,4
r1-net41-p1,p1,r1,net41,100,config,,32:FE:0F:35:B6:5F,enp8s0,enp8s0,None,4
r1-net12-p1,p1,r1,net12,100,config,,32:80:52:96:DF:29,enp7s0,enp7s0,None,4
r2-net2-p1,p1,r2,net2,100,config,,3A:52:D2:60:7D:8A,enp8s0,enp8s0,None,4



Time to print interfaces 4286 seconds


'3e42a97d-add1-4ec5-8c9c-f615afe41302'

In [3]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice ........................................................

In [60]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_conf = {
    "c1-net1-p1":  {"addr": "10.0.0.1", "subnet": "10.0.0.0/16"},
    "c2-net1-p1":  {"addr": "10.0.0.2", "subnet": "10.0.0.0/16"},
    "r1-net1-p1":  {"addr": "10.0.1.1", "subnet": "10.0.0.0/16"},
    
    "p1-net2-p1":  {"addr": "20.0.2.1", "subnet": "20.0.0.0/16"},
    "r2-net2-p1":  {"addr": "20.0.1.2", "subnet": "20.0.0.0/16"},
    
    "p2-net3-p1":  {"addr": "30.0.2.2", "subnet": "30.0.0.0/16"},
    "r3-net3-p1":  {"addr": "30.0.1.3", "subnet": "30.0.0.0/16"},
    
    "p3-net4-p1":  {"addr": "40.0.2.3", "subnet": "40.0.0.0/16"},
    "c3-net4-p1":  {"addr": "40.0.0.3", "subnet": "40.0.0.0/16"},
    "r4-net4-p1":  {"addr": "40.0.1.4", "subnet": "40.0.0.0/16"},
    
    "r1-net12-p1":  {"addr": "12.0.1.1", "subnet": "12.0.0.0/16"},
    "r2-net12-p1":  {"addr": "12.0.1.2", "subnet": "12.0.0.0/16"},
    
    "r2-net23-p1":  {"addr": "23.0.1.2", "subnet": "23.0.0.0/16"},
    "r3-net23-p1":  {"addr": "23.0.1.3", "subnet": "23.0.0.0/16"},
    
    "r3-net34-p1":  {"addr": "34.0.1.3", "subnet": "34.0.0.0/16"},
    "r4-net34-p1":  {"addr": "34.0.1.4", "subnet": "34.0.0.0/16"},
    
    "r4-net41-p1":  {"addr": "41.0.1.4", "subnet": "41.0.0.0/16"},
    "r1-net41-p1":  {"addr": "41.0.1.1", "subnet": "41.0.0.0/16"},
}

for iface in slice.get_interfaces():
    if_name = iface.get_name()
    iface.ip_addr_add(addr=if_conf[if_name]['addr'], subnet=IPv4Network(if_conf[if_name]['subnet']))
    
for n in ['r1','r2','r3','r4']:
    slice.get_node(name=n).execute("sudo sysctl -w net.ipv4.ip_forward=1")
    
for iface in slice.get_interfaces():
    iface.ip_link_up()
    

rt_conf = [
    {"name": "r1",  "addr": "20.0.0.0/16", "gw": "12.0.1.2"},
    {"name": "r1",  "addr": "40.0.0.0/16", "gw": "41.0.1.4"},
    {"name": "r1",  "addr": "30.0.0.0/16", "gw": "12.0.1.2"},
    
    {"name": "r2",  "addr": "10.0.0.0/16", "gw": "12.0.1.1"},
    {"name": "r2",  "addr": "30.0.0.0/16", "gw": "23.0.1.3"},
    {"name": "r2",  "addr": "40.0.0.0/16", "gw": "23.0.1.3"},
    
    {"name": "r3",  "addr": "20.0.0.0/16", "gw": "23.0.1.2"},
    {"name": "r3",  "addr": "40.0.0.0/16", "gw": "34.0.1.4"},
    {"name": "r3",  "addr": "10.0.0.0/16", "gw": "34.0.1.4"},
    
    {"name": "r4",  "addr": "10.0.0.0/16", "gw": "41.0.1.1"},
    {"name": "r4",  "addr": "30.0.0.0/16", "gw": "34.0.1.3"},
    {"name": "r4",  "addr": "20.0.0.0/16", "gw": "34.0.1.3"}
    
]

for rt in rt_conf:
    slice.get_node(name=rt['name']).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])
    
    
for n in ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]:
    slice.get_node(name=n).execute("sudo apt update; sudo apt -y install net-tools", quiet=True)
    
for n in ["c1","c2"]:
    slice.get_node(name=n).execute("sudo route add default gw 10.0.1.1 enp6s0", quiet=False)
for n in ["p1"]:
    slice.get_node(name=n).execute("sudo route add default gw 20.0.1.2 enp6s0", quiet=False)
for n in ["p2"]:
    slice.get_node(name=n).execute("sudo route add default gw 30.0.1.3 enp6s0", quiet=False)
for n in ["c3","p3"]:
    slice.get_node(name=n).execute("sudo route add default gw 40.0.1.4 enp6s0", quiet=False)
    
for n in ["r1"]:
    slice.get_node(name=n).execute("sudo route add default gw 40.0.1.4 enp6s0", quiet=False)
for n in ["r2"]:
    slice.get_node(name=n).execute("sudo route add default gw 10.0.1.1 enp7s0", quiet=False)
for n in ["r3"]:
    slice.get_node(name=n).execute("sudo route add default gw 20.0.1.2 enp7s0", quiet=False)
for n in ["r4"]:
    slice.get_node(name=n).execute("sudo route add default gw 30.0.1.3 enp6s0", quiet=False)
    


net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
 SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: Network is unreachable
  SIOCADDRT: Network is unreachable
  SIOCADDRT: No such device
 

,SSH command
Name,
c1,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fecc:f87c
c2,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe2d:7e45
c3,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe41:4337
p1,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe73:8775
p2,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe00:6911
p3,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fee1:94b6
r1,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fedc:5002
r2,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe63:9161
r3,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe09:378a


In [26]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
ssh_str = 'ssh -i ' + slice.get_slice_private_key_file() + \
    ' -J ' + fablib.get_bastion_username() + '@' + fablib.get_bastion_public_addr() + \
    ' -F /home/fabric/work/fabric_config/ssh_config '
slice_info = [{'Name': n.get_name(), 'SSH command': ssh_str + n.get_username() + '@' + str(n.get_management_ip())} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

,SSH command
Name,
c1,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fea9:d421
c2,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe08:8f64
c3,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fef3:9dc1
p1,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe6d:de91
p2,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe19:16ac
p3,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe1a:7763
r1,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe5f:5494
r2,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fec4:54fa
r3,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fed9:1c1e


Test a few connections through the network

In [63]:
slice.get_node(name="c1").execute("ping 30.0.2.2 -c 5", quiet=False)

 ping: connect: Network is unreachable
 

('', 'ping: connect: Network is unreachable\n')

In [62]:
slice.get_node(name="p1").execute("ping 40.0.2.3 -c 5", quiet=False)

 ping: connect: Network is unreachable
 

('', 'ping: connect: Network is unreachable\n')

Use this to extend the slice when needed

In [27]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 3 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

slice.update()
_ = slice.show()

ID,3e42a97d-add1-4ec5-8c9c-f615afe41302
Name,IP_NDN_Test_2_sjzhu
Lease Expiration (UTC),2023-12-04 03:35:27 +0000
Lease Start (UTC),2023-11-20 01:19:28 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


Use this section to delete the slice when needed

In [33]:
slice.delete()


In [34]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()

ID,3e42a97d-add1-4ec5-8c9c-f615afe41302
Name,IP_NDN_Test_2_sjzhu
Lease Expiration (UTC),2023-12-04 03:35:27 +0000
Lease Start (UTC),2023-11-20 01:19:28 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


Used for temp work 

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()
import os
slice_name="Project_Test_" + os.getenv('NB_USER')

slice = fablib.get_slice(slice_name)

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)

# Manual tweaking to limit node label overlap in the visualization
#pos["p2"] += (0.08, 0.0)
#pos["net3"] += (0.08, 0.0)


nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

NDN Setup:

In [28]:
hosts = ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]

FW_ARGS = {
  'mempool': {
    'DIRECT': { 'capacity': 524287, 'dataroom': 9146 },
    'INDIRECT': { 'capacity': 524287 },
  },
  'fib': {
    'capacity': 4095,
    'startDepth': 8,
  },
  'pcct': {
    'pcctCapacity': 65535,
    'csMemoryCapacity': 20000,
    'csIndirectCapacity': 20000,
  }
}
MEMIF_ARGS = {
    'scheme': "memif",
    'socketName': "/run/ndn/fileserver.sock",
    'id': 0,
    'role': "server",
    'dataroom': 9000
}

def make_fs_activate_json(node):
    activate_json = {
        'eal': {
            'memPerNuma': {'0': 4*1024},
            'filePrefix': 'producer',
        },
        'mempool': {
            'DIRECT': {'capacity': 2**16-1, 'dataroom': 9200},
            'INDIRECT': {'capacity': 2**16-1},
            'PAYLOAD': {'capacity': 2**16-1, 'dataroom': 9200},
        },
        'face': {
            'scheme': 'memif',
            'socketName': MEMIF_ARGS['socketName'],
            'id': MEMIF_ARGS['id'],
            'dataroom': 9000,
            'role': 'client',
        },
        'fileServer': {
            'mounts': [
                {'prefix': f"/{node}/usr-local-share", 'path': "/usr/local/share"}
            ],
            'segmentLen': 6 * 1024,
        },
    }
    return activate_json

net_to_nodes = {
    "net1": ["r1","c1","c2"],
    "net2": ["r2","p1"],
    "net3": ["r3","p2"],
    "net4": ["r4","p3","c3"],
    "net12": ["r1","r2"],
    "net23": ["r2","r3"],
    "net34": ["r3","r4"],
    "net41": ["r4","r1"],
}

adj_list = {
    "r1": ["c1","c2","r2","r4"],
    "r2": ["p1","r1","r3"],
    "r3": ["p2","r2","r4"],
    "r4": ["p3","c3","r1","r3"],
    "c1": ["r1"],
    "c2": ["r1"],
    "c3": ["r4"],
    "p1": ["r2"],
    "p2": ["r3"],
    "p3": ["r4"],
}
consumers = ["c1","c2","c3"]
routers = ["r1","r2","r3","r4"]
producers = ["p1", "p2", "p3"]

In [29]:
forwarder_hashes = {}
face_hashes = {}
node_net_interface = {}

In [30]:
import json
interfaces = json.loads(slice.list_interfaces(output="json"))
for i in interfaces:
    h = i["node"]
    net = i["network"]
    inf = i["physical_dev"]
    try:
        node_net_interface[h]
    except KeyError:
        node_net_interface[h] = {}
    node_net_interface[h][net] = inf
print(node_net_interface)

[
    {
        "name": "c1-net1-p1",
        "short_name": "p1",
        "node": "c1",
        "network": "net1",
        "bandwidth": "100",
        "mode": "config",
        "vlan": "",
        "mac": "0A:4C:01:D1:D8:41",
        "physical_dev": "enp6s0",
        "dev": "enp6s0",
        "ip_addr": "None",
        "numa": "4"
    },
    {
        "name": "c2-net1-p1",
        "short_name": "p1",
        "node": "c2",
        "network": "net1",
        "bandwidth": "100",
        "mode": "config",
        "vlan": "",
        "mac": "0E:47:AA:3F:E1:AA",
        "physical_dev": "enp6s0",
        "dev": "enp6s0",
        "ip_addr": "None",
        "numa": "4"
    },
    {
        "name": "c3-net4-p1",
        "short_name": "p1",
        "node": "c3",
        "network": "net4",
        "bandwidth": "100",
        "mode": "config",
        "vlan": "",
        "mac": "12:CC:D9:CC:0E:2F",
        "physical_dev": "enp6s0",
        "dev": "enp6s0",
        "ip_addr": "None",
        "numa": "

In [31]:
import shlex
import json

# add eth ports to each node
for h in (consumers + producers):
    node = slice.get_node(h)
    # consumers and producers only have 1 interface
    inf = next(iter(node_net_interface[h].values()))
    output = node.execute(f'''
        echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif {inf} --xdp --mtu 1500
        ''')
    forwarder_hashes[h] = json.loads(output[0].split("\n")[-2])

true
{"id":"7IEL0VV6B2794","isDown":false,"macAddr":"0a:4c:01:d1:d8:41","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"V4NKJODHK9IIC","isDown":false,"macAddr":"0e:47:aa:3f:e1:aa","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"DQMQCHKPOA234","isDown":false,"macAddr":"12:cc:d9:cc:0e:2f","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"5IMP3LNB3IDF4","isDown":false,"macAddr":"0e:a4:9f:77:13:42","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"KH9VQVG2D34IA","isDown":false,"macAddr":"12:fc:dc:55:68:10","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
 bash: line 2: ndndpdk-ctrl: command not found
bash: line 3: ndndpdk-ctrl: command not found
 

IndexError: list index out of range

In [32]:
print(consumers + producers)

['c1', 'c2', 'c3', 'p1', 'p2', 'p3']


In [69]:
for h in routers:
    forwarder_hashes[h] = {}
    for net, inf in iter(node_net_interface[h].items()):
        node = slice.get_node(h)
        output = node.execute(f'''
            echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
            ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif {inf} --xdp --mtu 1500
            ''')
        forwarder_hashes[h][net] = json.loads(output[0].split("\n")[-2])

true
{"id":"UCAADA618P57K","isDown":false,"macAddr":"1a:6c:1e:05:38:97","mtu":1500,"name":"net_af_xdp_4","numaSocket":null}
{"id":"UCAADA618P57M","isDown":false,"macAddr":"1e:ef:23:53:ab:f7","mtu":1500,"name":"net_af_xdp_5","numaSocket":null}
 2023/11/13 08:36:13 graphql: ndndpdk-svc is already activated
 {"id":"UCAADA618P57G","isDown":false,"macAddr":"1a:37:00:ed:dd:b7","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
 2023/11/13 08:36:16 graphql: ndndpdk-svc is already activated
 true
{"id":"55E9TANV69N2O","isDown":false,"macAddr":"12:4b:27:93:27:03","mtu":1500,"name":"net_af_xdp_5","numaSocket":null}
{"id":"55E9TANV69N2Q","isDown":false,"macAddr":"0e:73:c9:ed:13:af","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
 2023/11/13 08:36:21 graphql: ndndpdk-svc is already activated
 {"id":"55E9TANV69N2S","isDown":false,"macAddr":"0e:9f:4c:0b:29:47","mtu":1500,"name":"net_af_xdp_4","numaSocket":null}
 2023/11/13 08:36:23 graphql: ndndpdk-svc is already activated
 true
{"id":"77Q7SIG7

This cell is for caching results for creating the ethernet ports. DO NOT RUN THIS CELL WHEN RUNNING FROM SCRATCH

In [9]:
forwarder_hashes = {'c1': {'id': 'ACD2H53TMUI36', 'isDown': False, 'macAddr': '0a:52:8f:34:96:c6', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'c2': {'id': 'A2TMSRHUENN3I', 'isDown': False, 'macAddr': '0e:a1:da:68:b6:52', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'c3': {'id': 'HLLQQ4KG7NG6G', 'isDown': False, 'macAddr': '16:23:f2:ad:26:f0', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'p1': {'id': 'HUTK14PMB1LE6', 'isDown': False, 'macAddr': '12:6c:f4:55:67:15', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'p2': {'id': 'KAO1QT16OVJ3M', 'isDown': False, 'macAddr': '1a:7c:39:4f:e8:1c', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'p3': {'id': '0DUK5I1PE24BG', 'isDown': False, 'macAddr': '1e:51:b3:db:01:58', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'r1': {'net41': {'id': 'OJ3TV04CG8L4E', 'isDown': False, 'macAddr': '22:6a:06:25:64:06', 'mtu': 1500, 'name': 'net_af_xdp_4', 'numaSocket': None}, 'net12': {'id': 'OJ3TV04CG8L4C', 'isDown': False, 'macAddr': '26:39:f6:dc:cb:ce', 'mtu': 1500, 'name': 'net_af_xdp_5', 'numaSocket': None}, 'net1': {'id': 'OJ3TV04CG8L4A', 'isDown': False, 'macAddr': '1e:a6:b7:27:c2:17', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}}, 'r2': {'net12': {'id': 'U51QJCCGBG414', 'isDown': False, 'macAddr': '26:ec:72:3c:58:74', 'mtu': 1500, 'name': 'net_af_xdp_5', 'numaSocket': None}, 'net2': {'id': 'U51QJCCGBG416', 'isDown': False, 'macAddr': '26:9a:53:ec:da:62', 'mtu': 1500, 'name': 'net_af_xdp_4', 'numaSocket': None}, 'net23': {'id': 'U51QJCCGBG410', 'isDown': False, 'macAddr': '26:3e:26:77:3b:09', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}}, 'r3': {'net3': {'id': '8DQ0TTHACQ926', 'isDown': False, 'macAddr': '02:da:d0:de:84:4d', 'mtu': 1500, 'name': 'net_af_xdp_5', 'numaSocket': None}, 'net23': {'id': '8DQ0TTHACQ924', 'isDown': False, 'macAddr': '02:88:da:4f:7a:e8', 'mtu': 1500, 'name': 'net_af_xdp_4', 'numaSocket': None}, 'net34': {'id': '8DQ0TTHACQ922', 'isDown': False, 'macAddr': '02:7b:2e:86:34:1b', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}}, 'r4': {'net34': {'id': 'Q1EO5GI41VFIS', 'isDown': False, 'macAddr': '0a:07:1f:bd:98:8f', 'mtu': 1500, 'name': 'net_af_xdp_4', 'numaSocket': None}, 'net4': {'id': 'Q1EO5GI41VFIU', 'isDown': False, 'macAddr': '0a:1d:cc:d9:6d:0b', 'mtu': 1500, 'name': 'net_af_xdp_5', 'numaSocket': None}, 'net41': {'id': 'Q1EO5GI41VFIO', 'isDown': False, 'macAddr': '02:dc:a4:ab:d6:d8', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}}}
print(forwarder_hashes)

{'c1': {'id': 'ACD2H53TMUI36', 'isDown': False, 'macAddr': '0a:52:8f:34:96:c6', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'c2': {'id': 'A2TMSRHUENN3I', 'isDown': False, 'macAddr': '0e:a1:da:68:b6:52', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'c3': {'id': 'HLLQQ4KG7NG6G', 'isDown': False, 'macAddr': '16:23:f2:ad:26:f0', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'p1': {'id': 'HUTK14PMB1LE6', 'isDown': False, 'macAddr': '12:6c:f4:55:67:15', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'p2': {'id': 'KAO1QT16OVJ3M', 'isDown': False, 'macAddr': '1a:7c:39:4f:e8:1c', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'p3': {'id': '0DUK5I1PE24BG', 'isDown': False, 'macAddr': '1e:51:b3:db:01:58', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'r1': {'net41': {'id': 'OJ3TV04CG8L4E', 'isDown': False, 'macAddr': '22:6a:06:25:64:06', 'mtu': 1500, 'name': 'net_af_xdp_4', 'numaSocket': None}, 'net12': {'id': 'OJ3TV04CG8L4C

In [70]:
# this is a janky for loop that uses the routers to set up all the ethernet faces
for h in routers:
    node = slice.get_node(h)
    for net, output in iter(forwarder_hashes[h].items()):
        local_mac = output['macAddr']
        for r in net_to_nodes[net]:
            if h == r:
                continue
            else:
                print(f"h: {h}, r: {r}, net: {net}")
                print(f"local_mac: {local_mac}")
                if r in routers:
                    remote_mac = forwarder_hashes[r][net]['macAddr']
                    print(f"remote_mac: {remote_mac}")
                else:
                    remote_mac = forwarder_hashes[r]['macAddr']
                    print(f"remote_mac: {remote_mac}")
                    # do the reverse since we have the correct mac address here
                    remote_node = slice.get_node(r)
                    out2 = remote_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {remote_mac} --remote {local_mac}")
                    face_hashes[f"{r}:{h}"] = json.loads(out2[0])
                # we only do 1 way for routers because we're going to do the other side in another loop
                out = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
                face_hashes[f"{h}:{r}"] = json.loads(out[0])

h: r1, r: r2, net: net12
local_mac: 1a:6c:1e:05:38:97
remote_mac: 0e:9f:4c:0b:29:47
{"id":"U00QR2CU0H47KUQT"}
h: r1, r: c1, net: net1
local_mac: 1e:ef:23:53:ab:f7
remote_mac: 0e:b3:24:2c:eb:9e
{"id":"DK351UDGCC568JOG"}
{"id":"U00QR2CU0D4NQTIU"}
h: r1, r: c2, net: net1
local_mac: 1e:ef:23:53:ab:f7
remote_mac: 0e:a1:da:68:b6:52
{"id":"EV3HH4HIQKMMBQ6R"}
{"id":"U00QR2CU0L17KSAG"}
h: r1, r: r4, net: net41
local_mac: 1a:37:00:ed:dd:b7
remote_mac: 26:39:e4:86:61:05
{"id":"U00QR2CU0547UTQO"}
h: r2, r: p1, net: net2
local_mac: 12:4b:27:93:27:03
remote_mac: 0a:3c:b2:3f:7f:e2
{"id":"QILVMLSGQ8TM9JR9"}
{"id":"594PB2T0ELH2BL3G"}
h: r2, r: r3, net: net23
local_mac: 0e:73:c9:ed:13:af
remote_mac: 2a:b2:e7:c7:b8:a1
{"id":"594PB2T0EPGILL3I"}
h: r2, r: r1, net: net12
local_mac: 0e:9f:4c:0b:29:47
remote_mac: 1a:6c:1e:05:38:97
{"id":"594PB2T0ELGINKBL"}
h: r3, r: p2, net: net3
local_mac: 26:90:b6:cd:6f:20
remote_mac: 0a:53:98:c2:26:8e
{"id":"24CVUD0VILOTNI7G"}
{"id":"7BGNAQQO95DN8RB4"}
h: r3, r: r2, net: n

In [71]:
for p in producers:
    producer_node = slice.get_node(p)
    output = producer_node.execute(f'''
        MEMIF_FACE=$(echo {shlex.quote(json.dumps(MEMIF_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-face)
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh $(echo $MEMIF_FACE | jq -r .id)
        ''')
    # start fileserver instance
    output = producer_node.execute(f'''
        sudo ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/ systemd start
        echo {shlex.quote(json.dumps(make_fs_activate_json(p)))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3031 activate-fileserver
        ''')

{"id":"QIHVKTU4ITV2NJAOKLRLU"}
true
{"id":"248VS52BQCRPLHM1RF4SG"}
true
{"id":"BABUBE6L9T0L68IABABMS"}
true


This is for caching results from the interface creation, DO NOT RUN THIS CELL WHEN STARTING FROM SCRATCH

In [10]:
face_hashes = {'r1:r2': {'id': 'U00QR2CU0H47KUQT'}, 'c1:r1': {'id': 'DK351UDGCC568JOG'}, 'r1:c1': {'id': 'U00QR2CU0D4NQTIU'}, 'c2:r1': {'id': 'AEN6AJR167HJ3IRM'}, 'r1:c2': {'id': 'U00QR2CU053N6S2V'}, 'r1:r4': {'id': 'U00QR2CU0547UTQO'}, 'p1:r2': {'id': 'QILVMLSGQ8TM9JR9'}, 'r2:p1': {'id': '594PB2T0ELH2BL3G'}, 'r2:r3': {'id': '594PB2T0EPGILL3I'}, 'r2:r1': {'id': '594PB2T0ELGINKBL'}, 'p2:r3': {'id': '24CVUD0VILOTNI7G'}, 'r3:p2': {'id': '7BGNAQQO95DN8RB4'}, 'r3:r2': {'id': '7BGNAQQO9PBNURRC'}, 'r3:r4': {'id': '7BGNAQQO91CN4RB2'}, 'r4:r3': {'id': '5AF9B617BR8CEV7L'}, 'p3:r4': {'id': 'BAFU96411411I83G'}, 'r4:p3': {'id': '5AF9B617BN8SMTVI'}, 'c3:r4': {'id': 'FVASJPRSV8OK6EGK'}, 'r4:c3': {'id': '5AF9B617BRASCTNK'}, 'r4:r1': {'id': '5AF9B617BBFCAV8'}}
print(face_hashes)

{'r1:r2': {'id': 'U00QR2CU0H47KUQT'}, 'c1:r1': {'id': 'DK351UDGCC568JOG'}, 'r1:c1': {'id': 'U00QR2CU0D4NQTIU'}, 'c2:r1': {'id': 'AEN6AJR167HJ3IRM'}, 'r1:c2': {'id': 'U00QR2CU053N6S2V'}, 'r1:r4': {'id': 'U00QR2CU0547UTQO'}, 'p1:r2': {'id': 'QILVMLSGQ8TM9JR9'}, 'r2:p1': {'id': '594PB2T0ELH2BL3G'}, 'r2:r3': {'id': '594PB2T0EPGILL3I'}, 'r2:r1': {'id': '594PB2T0ELGINKBL'}, 'p2:r3': {'id': '24CVUD0VILOTNI7G'}, 'r3:p2': {'id': '7BGNAQQO95DN8RB4'}, 'r3:r2': {'id': '7BGNAQQO9PBNURRC'}, 'r3:r4': {'id': '7BGNAQQO91CN4RB2'}, 'r4:r3': {'id': '5AF9B617BR8CEV7L'}, 'p3:r4': {'id': 'BAFU96411411I83G'}, 'r4:p3': {'id': '5AF9B617BN8SMTVI'}, 'c3:r4': {'id': 'FVASJPRSV8OK6EGK'}, 'r4:c3': {'id': '5AF9B617BRASCTNK'}, 'r4:r1': {'id': '5AF9B617BBFCAV8'}}


In [73]:
# add FIB entries to point to producer from routers
for p in producers:
    for r in adj_list[p]:
        node = slice.get_node(r)
        c_to_r = face_hashes[f"{r}:{p}"]
        print(c_to_r)
        output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh {c_to_r['id']}")

{'id': '594PB2T0ELH2BL3G'}
{"id":"590P9AVK60J6BNQBELL6I"}
{'id': '7BGNAQQO95DN8RB4'}
{"id":"7BKN8IOC1OEJUP2TH66FG"}
{'id': '5AF9B617BN8SMTVI'}
{"id":"5AB99E3J32A8KTEB3MDM0"}


In [8]:
# add FIB entries to all consumers to point at attached routers
for c in consumers:
    for r in adj_list[c]:
        node = slice.get_node(c)
        c_to_r = face_hashes[f"{c}:{r}"]
        for p in producers:
            output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh {c_to_r['id']}")

{"id":"DK753MF44L6I8J9CORDVE"}
{"id":"DK753MF44L6I8J9CORDV8"}
{"id":"DK753MF44L6I8J9CORDVA"}
 2023/11/13 20:08:43 graphql: FIB unavailable
  2023/11/13 20:08:44 graphql: FIB unavailable
  2023/11/13 20:08:45 graphql: FIB unavailable
 {"id":"FVESHHP8NTQ0QDPE989D6"}
{"id":"FVESHHP8NTQ0QDPE989D0"}
{"id":"FVESHHP8NTQ0QDPE989D2"}


In [11]:
# add FIB entries to all routers to point to other routers for non-direct producers
for r in routers:
    node = slice.get_node(r)
    router_hops = []
    for h in adj_list[r]:
        if h in routers:
            router_hops.append(face_hashes[f"{r}:{h}"]['id'])
    for p in producers:
        if p in adj_list[r]:
            continue
        else:
            output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh {' --nh '.join(router_hops)}")

{"id":"U04QPAEA8G136UB0IMUDK"}
{"id":"U04QPAEA8G136UB0IMUDI"}
{"id":"U04QPAEA8G136UB0IMUDG"}
{"id":"590P9AVK60J6BNQBELL6K"}
{"id":"590P9AVK60J6BNQBELL6M"}
{"id":"7BKN8IOC1OEJUP2TH66FM"}
{"id":"7BKN8IOC1OEJUP2TH66FI"}
{"id":"5AB99E3J32A8KTEB3MDM4"}
{"id":"5AB99E3J32A8KTEB3MDM2"}
